In [1]:
import requests
import pandas as pd
import os

### Example URL request

In [2]:
url = "http://localhost:8983/solr/tech_products/select?indent=true&q.op=OR&q=viewsonic&rows=20&useParams="

In [3]:
response = requests.get(url)

In [4]:
response_json = response.json()
print(response_json)

{'responseHeader': {'status': 0, 'QTime': 0, 'params': {'q': 'viewsonic', 'indent': 'true', 'q.op': 'OR', 'rows': '20', 'useParams': ''}}, 'response': {'numFound': 1, 'start': 0, 'numFoundExact': True, 'docs': [{'id': 'VA902B', 'name': 'ViewSonic VA902B - flat panel display - TFT - 19"', 'manu': 'ViewSonic Corp.', 'manu_id_s': 'viewsonic', 'cat': ['electronics and stuff2'], 'features': ['19" TFT active matrix LCD, 8ms response time, 1280 x 1024 native resolution'], 'weight': 190.4, 'price': 279.95, 'price_c': '279.95,USD', 'popularity': 6, 'inStock': True, 'store': '45.18814,-93.88541', '_version_': 1795237645975027712, 'manu_exact': 'ViewSonic Corp.', 'price_c____l_ns': 27995, 'name_exact': 'ViewSonic VA902B - flat panel display - TFT - 19"'}]}}


### Processing the original crawled data

In [14]:
post_id_dictionary = {}

In [16]:
def convert_time(x):
    temp = x 
    if (isinstance(x,str)):
        temp = pd.to_datetime(temp,format= '%Y-%m-%d %H:%M:%S')
    solr_date = temp.strftime('%Y-%m-%dT%H:%M:%SZ')
    return solr_date

def ProcessSheetName(sheet_name):
    bk_name = sheet_name.split('(')[0]
    # remove empty space at the back if any
    if (bk_name[-1] == ' '):
        bk_name = bk_name[:-1]
    if (bk_name == 'GoT'):
        bk_name = "Game Of Thrones"
    elif (bk_name == 'LotR'):
        bk_name = "Lord of the Rings"

    return bk_name


def ProcessData(info_name):
    # info_name: Excel workbook name
    # post_csv_name: name of the csv file containing the posts of the books
    # comment_csv_name: name of the csv file containing the comments
    info = pd.ExcelFile(info_name)
    sheet_names = info.sheet_names
    post_dataframe = pd.read_excel(info, sheet_names[0])
    comment_dataframe = pd.read_excel(info, sheet_names[1])
    # add the book name
    bk_name_0 = ProcessSheetName(sheet_names[0])
    bk_name_1 = ProcessSheetName(sheet_names[1])
    bk_name_0 = pd.Series([bk_name_0] * len(post_dataframe.index))
    bk_name_1 = pd.Series([bk_name_1] * len(comment_dataframe.index))
    post_dataframe["book"] = bk_name_0
    comment_dataframe["book"] = bk_name_1

    for i in range(2, len(sheet_names)):
        raw_dataframe = pd.read_excel(info, sheet_names[i])

        # add book name
        bk_name = ProcessSheetName(sheet_names[i])

        bk_name = pd.Series([bk_name] * len(raw_dataframe.index))
        raw_dataframe["book"] = bk_name
        # Every even-indexed sheet is a POST sheet
        if (i%2 == 0):
            post_dataframe = pd.concat([post_dataframe, raw_dataframe])
        else:
            comment_dataframe = pd.concat([comment_dataframe, raw_dataframe])

    # label rows in the dataframe as POST or COMMENT for filtering when querying
    title = pd.Series(["POST"] * len(post_dataframe.index))
    post_dataframe["TYPE"] = title
    comment_title = pd.Series(["COMMENT"] * len(comment_dataframe.index))
    comment_dataframe["TYPE"] = comment_title

    # Convert time into pdate format as required by Solr
    post_created_utc = post_dataframe['created_utc']
    post_created_utc = post_created_utc.apply(convert_time)
    post_dataframe['created_utc'] = post_created_utc

    comment_created_utc = comment_dataframe['created_utc']
    comment_created_utc = comment_created_utc.apply(convert_time)
    comment_dataframe['created_utc'] = comment_created_utc

    return post_dataframe, comment_dataframe


def AddCommentID(row):
    if (row["post_id"] not in post_id_dictionary.keys()):
        post_id_dictionary[row["post_id"]] = 1
    else:
        post_id_dictionary[row["post_id"]] += 1
        
    return row["post_id"] + "_" + str(post_id_dictionary[row["post_id"]])
        
def AddCommentNum(row):
    if (row["post_id"] not in post_id_dictionary.keys()):
        post_id_dictionary[row["post_id"]] = 1
    else:
        post_id_dictionary[row["post_id"]] += 1
        
    return post_id_dictionary[row["post_id"]]

def ProcessCommentSheet(sheet_name):
    global post_id_dictionary
    post_id_dictionary = {}
    comment_df = pd.read_csv(sheet_name + ".csv")
    comment_df["id"] = comment_df.apply(AddCommentID, axis=1)

    # bring id column to the front
    cols = comment_df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    comment_df = comment_df[cols]
    
    # add comment num   
    post_id_dictionary = {}
    comment_df["comment_num"] = comment_df.apply(AddCommentNum, axis=1)
    comment_df.to_csv(sheet_name + "_process.csv",index=False)

### Custom function to process the initial data sheet

In [10]:
def ProcessDataInitial(info_name):
    # info_name: Excel workbook name
    # post_csv_name: name of the csv file containing the posts of the books
    # comment_csv_name: name of the csv file containing the comments
    info = pd.ExcelFile(info_name)
    sheet_names = info.sheet_names

    # for labelling categories later
    fantasy_cats = ["Game Of Thrones", "Lord of the Rings", "Harry Potter"]

    post_dataframe = pd.read_excel(info, sheet_names[0])
    comment_dataframe = pd.read_excel(info, sheet_names[1])
    # add the book name
    bk_name_0 = ProcessSheetName(sheet_names[0])
    bk_name_1 = ProcessSheetName(sheet_names[1])

    # label categories
    if (bk_name_0 in fantasy_cats):
        cat = pd.Series(["Fantasy"] * len(post_dataframe.index))
        post_dataframe["category"] = cat
    else:
        cat = pd.Series(["Romance"] * len(post_dataframe.index))
        post_dataframe["category"] = cat

    if (bk_name_1 in fantasy_cats):
        cat = pd.Series(["Fantasy"] * len(comment_dataframe.index))
        comment_dataframe["category"] = cat
    else:
        cat = pd.Series(["Romance"] * len(comment_dataframe.index))
        comment_dataframe["category"] = cat
        
    bk_name_0 = pd.Series([bk_name_0] * len(post_dataframe.index))
    bk_name_1 = pd.Series([bk_name_1] * len(comment_dataframe.index))
    post_dataframe["book"] = bk_name_0
    comment_dataframe["book"] = bk_name_1

    

    
    for i in range(2, len(sheet_names)):
        raw_dataframe = pd.read_excel(info, sheet_names[i])

        # add book name
        bk_name = ProcessSheetName(sheet_names[i])

        # add category
        if (bk_name in fantasy_cats):
            cat = pd.Series(["Fantasy"] * len(raw_dataframe.index))
            raw_dataframe["category"] = cat
        else:
            cat = pd.Series(["Romance"] * len(raw_dataframe.index))
            raw_dataframe["category"] = cat

        # add book name
        bk_name = pd.Series([bk_name] * len(raw_dataframe.index))
        raw_dataframe["book"] = bk_name
        

        # Every even-indexed sheet is a POST sheet
        if (i%2 == 0):
            post_dataframe = pd.concat([post_dataframe, raw_dataframe])
        else:
            comment_dataframe = pd.concat([comment_dataframe, raw_dataframe])

    # label rows in the dataframe as POST or COMMENT for filtering when querying
    title = pd.Series(["POST"] * len(post_dataframe.index))
    post_dataframe["TYPE"] = title
    comment_title = pd.Series(["COMMENT"] * len(comment_dataframe.index))
    comment_dataframe["TYPE"] = comment_title

    # Convert time into pdate format as required by Solr
    post_created_utc = post_dataframe['created_utc']
    post_created_utc = post_created_utc.apply(convert_time)
    post_dataframe['created_utc'] = post_created_utc

    comment_created_utc = comment_dataframe['created_utc']
    comment_created_utc = comment_created_utc.apply(convert_time)
    comment_dataframe['created_utc'] = comment_created_utc

    return post_dataframe, comment_dataframe

### Process the first two sheets

In [17]:
post_df, comment_df = ProcessDataInitial("Info.xlsx")
post_df.to_csv("post.csv", index=False)
comment_df.to_csv("comment.csv", index=False)

ProcessCommentSheet("comment")

In [6]:
data_dir = "./Book Data/"
excel_names = ["Action Books.xlsx", "Comedy Books.xlsx", "Fantasy Books.xlsx", "Horror Books.xlsx", "Mystery Books.xlsx"]
comment_names = []


for i in excel_names:
    post_df, comment_df = ProcessData(data_dir+i)

    # Adding category
    cat_name_post = pd.Series([i.split()[0]] * len(post_df.index))
    cat_name_comment = pd.Series([i.split()[0]] * len(comment_df.index))
    post_df['category'] = cat_name_post
    comment_df['category'] = cat_name_comment

    # Saving csv
    post_df.to_csv(i.split()[0] + "_post.csv", index=False)
    comment_df.to_csv(i.split()[0] + "_comment.csv", index=False)

    # Record comment sheets to process comment id and num later
    comment_names.append(i.split()[0] + "_comment")


### Add Comment ID and Comment num to comment data sheets

In [7]:
post_id_dictionary = {}
for i in comment_names:
    ProcessCommentSheet(i)

Action_Dune_analyzed_reviews.xlsx
Action_Jurassic Park_analyzed_reviews.xlsx
Action_Ready Player One_analyzed_reviews.xlsx
Action_The Hunger Games_analyzed_reviews.xlsx
Action_The Maze Runner_analyzed_reviews.xlsx
Comedy_Catch-22_analyzed_reviews.xlsx
Comedy_Confed of Dunces_analyzed_reviews.xlsx
Comedy_Good Omens_analyzed_reviews.xlsx
Comedy_The Catcher in the Rye_analyzed_reviews.xlsx
Comedy_The Idiot_analyzed_reviews.xlsx
Fantasy_Eragon_analyzed_reviews.xlsx
Fantasy_Narnia_analyzed_reviews.xlsx
Fantasy_The Hobbit_analyzed_reviews.xlsx
Fantasy_The Last Wish_analyzed_reviews.xlsx
Fantasy_The Royal Ranger_analyzed_reviews.xlsx
Horror_Dracula_analyzed_reviews.xlsx
Horror_Frankenstein_analyzed_reviews.xlsx
Horror_THe Exorcist_analyzed_reviews.xlsx
Horror_The Housemaid_analyzed_reviews.xlsx
Horror_The Shining_analyzed_reviews.xlsx
Mystery_Devil in the White City_analyzed_reviews.xlsx
Mystery_Gone Girl_analyzed_reviews.xlsx
Mystery_Rebecca_analyzed_reviews.xlsx
Mystery_The Maid_analyzed_re